In [54]:
import struct

In [55]:
log = 'sample_log.txt'

In [56]:
log_b[:20]

b'\x01\x00\x00\x00J+\x00\x00\xfe\xff\xfb\xff\xfc\xff\x84\xff\x0b\x00\x01\x04'

In [57]:
log_b = None
with open(log, 'r') as f:
    log_str = f.read()
    log_b = bytes.fromhex(log_str)

In [58]:
print(log_b[:20])

b'\x07\x00\x00\x00\xaca\x00\x00o\x12\x03<5^\xba=\x01\x00\x00\x00'


In [75]:
i = 0
while i < len(log_b):
    t = struct.unpack('<L', log_b[i:i+4])[0]
    i += 4
    if t in [1, 2, 3]:
        ts, gyro_x, gyro_y, gyro_z, acc_x, acc_y, acc_z = struct.unpack('<Lhhhhhh', log_b[i: i+16])
        print(f"{ts} IMU {t%3}: Gx: {gyro_x}, Gy: {gyro_y}, Gz: {gyro_z}, Ax: {acc_x}, Ay: {acc_y}, Az: {acc_z}")
        #print(imu_data)
        i += 4 + 6 + 6 #ts + 6 x int16
    elif t in [4, 5, 6]:
        ts, pressure, temperature = struct.unpack('<LLL', log_b[i: i+12])
        print(f"{ts} BARO {t%3}: P: {pressure}, T: {temperature}")     
        #print(baro_data)
        i += 4 + 4 + 4
    elif t == 7:
        ts, height, velocity = struct.unpack('<Lff', log_b[i: i+12])
        print(f"{ts} FLIGHT_INFO: Height: {height}, Velocity: {velocity}")
        #print(flight_info_data)
        i += 4 + 4 + 4
    elif t == 8:
        print("FLIGHT_STATE")
        flight_state_data = struct.unpack('<LB', log_b[i: i+5])
        i += 4 + 1
    else:
        print("ERROR")

25004 FLIGHT_INFO: Height: 0.00800000037997961, Velocity: 0.09099999815225601
25006 IMU 1: Gx: -2, Gy: -6, Gz: -3, Ax: -71, Ay: -84, Az: 1013
25009 IMU 2: Gx: 26, Gy: -24, Gz: -2, Ax: -66, Ay: -71, Az: 1022
25011 BARO 1: P: 95382, T: 2847
25011 BARO 2: P: 95491, T: 2781
25011 BARO 0: P: 95567, T: 2786
25012 IMU 0: Gx: 8, Gy: 3, Gz: -8, Ax: -70, Ay: -84, Az: 1023
25014 FLIGHT_INFO: Height: 0.012000000104308128, Velocity: 0.06800000369548798
25015 IMU 1: Gx: -2, Gy: -5, Gz: -3, Ax: -71, Ay: -84, Az: 1013
25018 IMU 2: Gx: 25, Gy: -25, Gz: -2, Ax: -66, Ay: -70, Az: 1021
25021 IMU 0: Gx: 8, Gy: 2, Gz: -8, Ax: -71, Ay: -84, Az: 1022
25021 BARO 1: P: 95341, T: 2847
25021 BARO 2: P: 95470, T: 2781
25021 BARO 0: P: 95588, T: 2786
25024 IMU 1: Gx: -2, Gy: -6, Gz: -4, Ax: -71, Ay: -84, Az: 1012
25024 FLIGHT_INFO: Height: 0.006000000052154064, Velocity: 0.07500000298023224
25027 IMU 2: Gx: 25, Gy: -24, Gz: -2, Ax: -67, Ay: -69, Az: 1023
25031 BARO 1: P: 95341, T: 2847
25031 BARO 2: P: 95470, T: 27

error: unpack requires a buffer of 12 bytes